In [1]:
import os
#GPU_NO = "3"
#os.environ["CUDA_VISIBLE_DEVICES"] = GPU_NO
TRANSFORMERS_CACHE='/workspace/HF_cache/transformers_cache/'

In [2]:
import sys
MGIT_PATH=os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(MGIT_PATH)
from utils.lineage.graph import *
from utils import meta_functions

In [3]:
success_condition_path=os.getcwd()+'/user_functions.py'
success_condition_name='test_success_condition'

In [4]:
compute_metrics_path=os.getcwd()+'/user_functions.py'
compute_metrics_name='compute_metrics'

In [5]:
lineage_eval_dataset = LineageDataset("glue", "sst2", split="validation", cache_dir=TRANSFORMERS_CACHE, feature_keys=['sentence'])

In [6]:
#lineage_train_dataset = LineageDataset("glue", "sst2", split="train", cache_dir=TRANSFORMERS_CACHE, feature_keys=['premise','hypothesis'])
lineage_train_dataset = LineageDataset("glue", "sst2", split="validation", cache_dir=TRANSFORMERS_CACHE, feature_keys=['sentence'])

In [7]:
preprocess_file = os.path.join(MGIT_PATH,'utils/preprocess_utils.py')
preprocess_function = 'glue_preprocess_function'

In [8]:
perturbation_file = os.path.join(MGIT_PATH,'utils/perturbations/perturbation_utils.py')
perturbation_name = 'perturb_char_misspelledword'

In [9]:
!rm -rf tmp_sst2_node1
!rm -rf tmp_sst2_node2
!rm -rf parameter_store

In [9]:
g = LineageGraph()

In [10]:
lineage_train = LineageTrain(
    preprocess_function_path=preprocess_file,
    preprocess_function_name=preprocess_function,
    train_dataset=lineage_train_dataset,
    eval_dataset=lineage_eval_dataset,
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
)
node1 = LineageNode(
    init_checkpoint='roberta-base',
    lineage_train=lineage_train,
    output_dir='tmp_sst2_node1',
)

g.add(node1)

tmp_sst2_node1 becomes the root, as it is the first inserted node.


True

In [11]:
lineage_train = LineageTrain(
    preprocess_function_path=preprocess_file,
    preprocess_function_name=preprocess_function,
    train_dataset=lineage_train_dataset,
    eval_dataset=lineage_eval_dataset,
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
)
node2 = LineageNode(
    init_checkpoint='roberta-base',
    model_init_function_path=success_condition_path,
    model_init_function_name='vanilla_finetune_init_function',
    lineage_train=lineage_train,
    output_dir='tmp_sst2_node2',
    is_delta=True,
)

g.add(node2,etype='adapted',parent='tmp_sst2_node1')
g.add(node2,etype='versioned',parent='tmp_sst2_node1')

True

In [12]:
lineage_train = LineageTrain(
    preprocess_function_path=preprocess_file,
    preprocess_function_name=preprocess_function,
    train_dataset=lineage_train_dataset,
    eval_dataset=lineage_eval_dataset,
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
)
node3 = LineageNode(
    init_checkpoint='roberta-base',
    model_init_function_path=success_condition_path,
    model_init_function_name='vanilla_finetune_init_function',
    lineage_train=lineage_train,
    output_dir='tmp_sst2_node3',
    is_delta=True,
)

g.add(node3,etype='adapted',parent='tmp_sst2_node2')
g.add(node3,etype='versioned',parent='tmp_sst2_node2')

True

In [13]:
test1 = LineageTest(
        preprocess_function_path=preprocess_file,
        preprocess_function_name=preprocess_function,        
        eval_dataset=lineage_eval_dataset,
        test_success_condition_path=success_condition_path,
        test_success_condition_name=success_condition_name,
        compute_metrics_path=compute_metrics_path,
        compute_metrics_name=compute_metrics_name,
        metric_for_best_model='accuracy',
        name='test1',
)

In [14]:
node1.add_test(test1)

In [15]:
node2.add_test(test1)

In [16]:
node3.add_test(test1)

In [17]:
test2 = LineageTest(
        perturbation_function_path=perturbation_file,
        perturbation_function_name=perturbation_name,
        preprocess_function_path=preprocess_file,
        preprocess_function_name=preprocess_function,
        eval_dataset=lineage_eval_dataset,
        test_success_condition_path=success_condition_path,
        test_success_condition_name=success_condition_name,
        compute_metrics_path=compute_metrics_path,
        compute_metrics_name=compute_metrics_name,
        metric_for_best_model='accuracy',
        name='test2',
)

In [18]:
node1.add_test(test2)

In [19]:
node2.add_test(test2)

In [20]:
node3.add_test(test2)

In [22]:
node1.train()
node1.run_all_tests()

loading model: roberta-base


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, org_sentence, sentence. If idx, org_sentence, sentence are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 852
  Batch size = 8


{'eval_loss': 0.5311130881309509, 'eval_accuracy': 0.7828638497652582, 'eval_runtime': 27.1308, 'eval_samples_per_second': 31.403, 'eval_steps_per_second': 3.944}
True


[True, True]

In [23]:
node2.train()
node2.run_all_tests()

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading configuration file https://huggingface.co/roberta-

loading model: roberta-base


loading weights file https://huggingface.co/roberta-base/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (ini

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, org_sentence, sentence. If idx, org_sentence, sentence are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 852
  Batch size = 8


{'eval_loss': 0.27072569727897644, 'eval_accuracy': 0.9096244131455399, 'eval_runtime': 25.7512, 'eval_samples_per_second': 33.086, 'eval_steps_per_second': 4.155}
True


[True, True]

In [24]:
node3.train()
node3.run_all_tests()

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading configuration file https://huggingface.co/roberta-

loading model: roberta-base


loading weights file https://huggingface.co/roberta-base/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (ini

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 8


{'eval_loss': 0.045914798974990845, 'eval_accuracy': 0.9862385321100917, 'eval_runtime': 19.2733, 'eval_samples_per_second': 45.244, 'eval_steps_per_second': 5.656}
True
                

OSError: [Errno 12] Cannot allocate memory

In [25]:
all_nodes = ['tmp_sst2_node1','tmp_sst2_node2','tmp_sst2_node3']
meta_functions.show_result_table(g,all_nodes)

+----------------+-------+-------+
| nodes/tests    | test1 | test2 |
+----------------+-------+-------+
| tmp_sst2_node1 | True  | True  |
+----------------+-------+-------+
| tmp_sst2_node2 | True  | True  |
+----------------+-------+-------+
| tmp_sst2_node3 | True  | True  |
+----------------+-------+-------+


In [26]:
meta_functions.show_result_table(g,all_nodes,show_metrics=True)

+----------------+-------+-------+
| nodes/tests    | test1 | test2 |
+----------------+-------+-------+
| tmp_sst2_node1 | 0.876 | 0.783 |
+----------------+-------+-------+
| tmp_sst2_node2 | 0.972 | 0.910 |
+----------------+-------+-------+
| tmp_sst2_node3 | 0.986 | 0.939 |
+----------------+-------+-------+


In [27]:
assert node1.is_test_failure() is False

In [28]:
assert node2.is_test_failure() is False

In [29]:
assert node3.is_test_failure() is False

In [33]:
node1.unload_model()
node2.unload_model()
node3.unload_model()

In [34]:
node1.run_all_tests()

loading configuration file tmp_sst2_node1/config.json
Model config RobertaConfig {
  "_name_or_path": "tmp_sst2_node1",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading configuration file tmp_sst2_node1/config.json
Model config RobertaConfig {
  "_name_or_path": "tmp_sst2_node1",
  "architectures": [
    "RobertaForSequenceClassification"
 

loading model: tmp_sst2_node1


All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at tmp_sst2_node1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.
Didn't find file tmp_sst2_node1/added_tokens.json. We won't load it.
loading file tmp_sst2_node1/vocab.json
loading file tmp_sst2_node1/merges.txt
loading file tmp_sst2_node1/tokenizer.json
loading file None
loading file tmp_sst2_node1/special_tokens_map.json
loading file tmp_sst2_node1/tokenizer_config.json


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: org_sentence, sentence, idx. If org_sentence, sentence, idx are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 852
  Batch size = 8


{'eval_loss': 0.5311130881309509, 'eval_accuracy': 0.7828638497652582, 'eval_runtime': 27.2526, 'eval_samples_per_second': 31.263, 'eval_steps_per_second': 3.926}
True


[True, True]

In [35]:
node2.run_all_tests()

loading configuration file tmp_sst2_node2/config.json
Model config RobertaConfig {
  "_name_or_path": "tmp_sst2_node2",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading configuration file tmp_sst2_node2/config.json
Model config RobertaConfig {
  "_name_or_path": "tmp_sst2_node2",
  "architectures": [
    "RobertaForSequenceClassification"
 

loading model: tmp_sst2_node2


All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at tmp_sst2_node2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.
Didn't find file tmp_sst2_node2/added_tokens.json. We won't load it.
loading file tmp_sst2_node2/vocab.json
loading file tmp_sst2_node2/merges.txt
loading file tmp_sst2_node2/tokenizer.json
loading file None
loading file tmp_sst2_node2/special_tokens_map.json
loading file tmp_sst2_node2/tokenizer_config.json


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: org_sentence, sentence, idx. If org_sentence, sentence, idx are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 852
  Batch size = 8


{'eval_loss': 0.23859763145446777, 'eval_accuracy': 0.9213615023474179, 'eval_runtime': 26.7516, 'eval_samples_per_second': 31.849, 'eval_steps_per_second': 4.0}
True


[True, True]

In [36]:
node3.run_all_tests()

loading configuration file tmp_sst2_node3/config.json
Model config RobertaConfig {
  "_name_or_path": "tmp_sst2_node3",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading configuration file tmp_sst2_node3/config.json
Model config RobertaConfig {
  "_name_or_path": "tmp_sst2_node3",
  "architectures": [
    "RobertaForSequenceClassification"
 

loading model: tmp_sst2_node3


All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at tmp_sst2_node3.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.
Didn't find file tmp_sst2_node3/added_tokens.json. We won't load it.
loading file tmp_sst2_node3/vocab.json
loading file tmp_sst2_node3/merges.txt
loading file tmp_sst2_node3/tokenizer.json
loading file None
loading file tmp_sst2_node3/special_tokens_map.json
loading file tmp_sst2_node3/tokenizer_config.json


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: org_sentence, sentence, idx. If org_sentence, sentence, idx are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 852
  Batch size = 8


{'eval_loss': 0.2463987171649933, 'eval_accuracy': 0.9389671361502347, 'eval_runtime': 27.1764, 'eval_samples_per_second': 31.351, 'eval_steps_per_second': 3.937}
True


[True, True]

In [38]:
all_nodes = ['tmp_sst2_node1','tmp_sst2_node2','tmp_sst2_node3']

In [39]:
meta_functions.show_result_table(g,all_nodes,show_metrics=True)

+----------------+-------+-------+
| nodes/tests    | test2 | test1 |
+----------------+-------+-------+
| tmp_sst2_node1 | 0.783 | 0.876 |
+----------------+-------+-------+
| tmp_sst2_node2 | 0.921 | 0.974 |
+----------------+-------+-------+
| tmp_sst2_node3 | 0.939 | 0.987 |
+----------------+-------+-------+


In [ ]:
for k, v in node1.get_pt_model().state_dict().items():
    delta = node2.get_pt_model().state_dict()[k] - v
    print(delta)

In [54]:
res = !du -s tmp_sst2_node1
node1_store = int(res[0].split('\t')[0])

res = !du -s tmp_sst2_node2
node2_store = int(res[0].split('\t')[0])

res = !du -s tmp_sst2_node3
node3_store = int(res[0].split('\t')[0])

res = !du -s parameter_store
global_store = int(res[0].split('\t')[0])

print('storage savings:',(node1_store+node2_store+node3_store)/global_store)

storage savings: 2.8191782081841943
